In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, CuDNNGRU, Dropout, BatchNormalization, Activation

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
data = pd.read_csv('train.csv')
# Keeping only the neccessary columns
data = data[['Tweet','Category']]
data.head()

,Tweet,Category
0,IOS 9 App Transport Security. Mm need to check...,neutral
1,"Mar if you have an iOS device, you should down...",neutral
2,@jimmie_vanagon my phone does not run on lates...,negative
3,Not sure how to start your publication on iOS?...,positive
4,"Two Dollar Tuesday is here with Forklift 2, Qu...",neutral


In [3]:
data[:5]

,Tweet,Category
0,IOS 9 App Transport Security. Mm need to check...,neutral
1,"Mar if you have an iOS device, you should down...",neutral
2,@jimmie_vanagon my phone does not run on lates...,negative
3,Not sure how to start your publication on iOS?...,positive
4,"Two Dollar Tuesday is here with Forklift 2, Qu...",neutral


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5422 entries, 0 to 5421
Data columns (total 2 columns):
Tweet       5422 non-null object
Category    5422 non-null object
dtypes: object(2)
memory usage: 84.8+ KB


In [5]:
data.describe()

,Tweet,Category
count,5422,5422
unique,5417,4
top,@pperrin @nellesapelle Very wise. Hard to see ...,positive
freq,2,2599


In [6]:
data.columns

Index(['Tweet', 'Category'], dtype='object')

In [7]:
data.dtypes

Tweet       object
Category    object
dtype: object

In [8]:
data.isnull().sum()

Tweet       0
Category    0
dtype: int64

In [9]:
print(data.size)
data[:5]

10844


,Tweet,Category
0,IOS 9 App Transport Security. Mm need to check...,neutral
1,"Mar if you have an iOS device, you should down...",neutral
2,@jimmie_vanagon my phone does not run on lates...,negative
3,Not sure how to start your publication on iOS?...,positive
4,"Two Dollar Tuesday is here with Forklift 2, Qu...",neutral


In [10]:
# it will remove all Neutral values from data
data = data[data.Category != "neutral"]
print(data[:2])

# it will remove all the eg:-  RT @NancyLeeGrahn:  
data['Tweet'] = data['Tweet'].apply(lambda x: x.lower())
print(data[:2])
#print(data)
data['Tweet'] = data['Tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
print(data[:2])
print(data.dtypes)
print(data[ data['Category'] == 'positive'].size)
print(data[ data['Category'] == 'negative'].size)
print(data[data['Category']!='neutral'].size)

for idx,row in data.iterrows():
    #print(idx,row)
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Tweet'].values)
X = tokenizer.texts_to_sequences(data['Tweet'].values)
print(X[:5])


# pad: to make all input of same length
X = pad_sequences(X)
print(X[:5])

                                               Tweet  Category
2  @jimmie_vanagon my phone does not run on lates...  negative
3  Not sure how to start your publication on iOS?...  positive
                                               Tweet  Category
2  @jimmie_vanagon my phone does not run on lates...  negative
3  not sure how to start your publication on ios?...  positive
                                               Tweet  Category
2  jimmie_vanagon my phone does not run on latest...  negative
3  not sure how to start your publication on ios ...  positive
Tweet       object
Category    object
dtype: object
5198
1738
6938
[[17, 240, 282, 31, 303, 5, 1265, 236, 293, 13, 979, 10, 703, 1, 334, 22, 34, 14, 24], [31, 290, 108, 2, 98, 54, 5, 236, 140, 12, 159, 15, 540, 39, 905, 66, 6, 46], [470, 10, 236, 1837, 335, 20, 11, 156, 241, 115, 470, 1838, 8, 654, 42, 291, 10, 1489, 303], [66, 1839, 15, 830, 10, 236, 6, 704, 1490, 7, 1, 70, 6, 390, 130, 9, 59, 11, 151, 2, 82], [88, 4, 1491, 11, 

In [11]:
# Keras offers an Embedding layer that can be used for neural networks on text data.
'''
source from: machinelearningmastery.com
The Embedding layer is defined as the first hidden layer of a network. It must specify 3 arguments:

It must specify 3 arguments:

input_dim: This is the size of the vocabulary in the text data. For example, if your data is integer encoded to values between 0-10, then the size of the vocabulary would be 11 words.
output_dim: This is the size of the vector space in which words will be embedded. It defines the size of the output vectors from this layer for each word. For example, it could be 32 or 100 or even larger. Test different values for your problem.
input_length: This is the length of input sequences, as you would define for any input layer of a Keras model. For example, if all of your input documents are comprised of 1000 words, this would be 1000.
For example, below we define an Embedding layer with a vocabulary of 200 (e.g. integer encoded words from 0 to 199, inclusive), a vector space of 32 dimensions in which words will be embedded, and input documents that have 50 words each.


e = Embedding(200, 32, input_length=50)
1
e = Embedding(200, 32, input_length=50)


'''
embed_dim = 128
lstm_out = 196
import time
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
start=time.time()
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print("Time to compile model:",time.time()-start)
print(model.summary())

W0729 19:58:17.789808 4548240832 deprecation_wrapper.py:119] From /Users/valarmathipukuraj/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0729 19:58:17.813063 4548240832 deprecation_wrapper.py:119] From /Users/valarmathipukuraj/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0729 19:58:17.819224 4548240832 deprecation_wrapper.py:119] From /Users/valarmathipukuraj/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0729 19:58:17.896563 4548240832 deprecation_wrapper.py:119] From /Users/valarmathipukuraj/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Pleas

Time to compile model: 0.06422305107116699
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 32, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
print((data['Category']).values)
Y = pd.get_dummies(data['Category']).values
print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

['negative' 'positive' 'positive' ... 'negative' 'negative' 'positive']
[[0 1 0]
 [0 0 1]
 [0 0 1]
 ...
 [0 1 0]
 [0 1 0]
 [0 0 1]]
(2948, 32) (2948, 3)
(521, 32) (521, 3)


In [13]:

batch_size = 32
model.fit(X_train, Y_train, batch_size = 64, epochs =50, verbose=1, validation_split=0.01, shuffle=True)

W0729 19:58:30.677573 4548240832 deprecation.py:323] From /Users/valarmathipukuraj/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2918 samples, validate on 30 samples
Epoch 1/50
2918/2918 [==============================] - 6s 2ms/step - loss: 0.6885 - acc: 0.7416 - val_loss: 0.4988 - val_acc: 0.8000
Epoch 2/50
2918/2918 [==============================] - 5s 2ms/step - loss: 0.5375 - acc: 0.7543 - val_loss: 0.4588 - val_acc: 0.8000
Epoch 3/50
2918/2918 [==============================] - 5s 2ms/step - loss: 0.4287 - acc: 0.8033 - val_loss: 0.3552 - val_acc: 0.8333
Epoch 4/50
2918/2918 [==============================] - 5s 2ms/step - loss: 0.3073 - acc: 0.8715 - val_loss: 0.3439 - val_acc: 0.9333
Epoch 5/50
2918/2918 [==============================] - 5s 2ms/step - loss: 0.2530 - acc: 0.8965 - val_loss: 0.3774 - val_acc: 0.9333
Epoch 6/50
2918/2918 [==============================] - 5s 2ms/step - loss: 0.2034 - acc: 0.9239 - val_loss: 0.4595 - val_acc: 0.8333
Epoch 7/50
2918/2918 [==============================] - 5s 2ms/step - loss: 0.1744 - acc: 0.9315 - val_loss: 0.4348 - val_acc: 0.8667
Epoch 8/50
2918/

In [14]:
# validation_size = 1500

# X_validate = X_test[-validation_size:]
# Y_validate = Y_test[-validation_size:]
# X_test = X_test[:-validation_size]
# Y_test = Y_test[:-validation_size]
results = model.evaluate(X_test, Y_test, batch_size=128)
print('test loss, test acc:', results)


521/521 [==============================] - 0s 540us/step
test loss, test acc: [1.8508711250180705, 0.7850287910157568]


In [15]:
score = model.evaluate(X_test, Y_test)

print ("Test Loss = " + str(score[0]))
print ("Test Accuracy = " + str(score[1]))

521/521 [==============================] - 0s 497us/step
Test Loss = 1.850871168949325
Test Accuracy = 0.7850287910157568


In [16]:
model.save("twitter_model_trained.h5")

In [17]:
from keras.models import load_model
twitter_model = load_model("twitter_model_trained.h5")

In [18]:
model_loss, model_accuracy = twitter_model.evaluate(
    X_test, Y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 1.850871168949325, Accuracy: 0.7850287910157568


In [ ]:
done = False
while done == False:
    sentence = input('Input sentence: ')
  
    sentence_encoding = tokenizer.texts_to_sequences([sentence])
    padded_sentence = sequence.pad_sequences(sentence_encoding, maxlen=32)
  
    prediction = model.predict(np.array(padded_sentence))
    print(max(prediction))
    if prediction[0][0] == max(prediction[0]):
#         print(prediction[0])
        print('Positive')
    elif prediction[0][1] == max(prediction[0]):
#         print(prediction[0])
        print('neutral')
    elif prediction[0][2] == max(prediction[0]):
#         print(prediction[0])
        print('negative')

    
  
    finish = input('Do you still want to input another text? [y/n]')
    if finish == 'N' or finish == 'n':
        done = True

Input sentence: Why would you waste a good iPod like this...idiots.
[5.7355708e-05 3.0966074e-04 9.9963295e-01]
negative
Do you still want to input another text? [y/n]y
Input sentence:  my birthday is this coming Thursday and I don't necessarily want an Ipod but a happy birthday would suffice haha
[4.1951375e-06 7.9003265e-05 9.9991679e-01]
negative
Do you still want to input another text? [y/n]y
Input sentence: happy
[0.66274124 0.07626074 0.260998  ]
Positive
Do you still want to input another text? [y/n]y
Input sentence: haha
[0.0379269  0.01192971 0.9501434 ]
negative
Do you still want to input another text? [y/n]No iPod until 10th grade began... Brother is still in elementary and succeeded in convincing dad he 'needs' one.
Input sentence: y
[0.00313517 0.00267506 0.99418974]
negative
Do you still want to input another text? [y/n]No iPod until 10th grade began... Brother is still in elementary and succeeded in convincing dad he 'needs' one.
Input sentence: No iPod until 10th grade 